# Tools - File Search

This lesson provides a hands-on guide to using the **File Search** feature in OpenAI's Assistants API. Learn how to empower your assistants with the ability to search and retrieve information from your files, making them significantly more knowledgeable and useful.

By the end of this lesson, you will be able to:

1.  Understand the core concepts of File Search and **Vector Stores**.
2.  Upload files and create Vector Stores to manage your assistant's knowledge base.
3.  Integrate File Search into your Assistants and Threads.
4.  Customize File Search settings for optimal performance.
5.  Understand cost management with expiration policies.

## Introduction

**File Search** lets your assistant tap into a wealth of information beyond its base model's training data. By uploading your files, your assistant can leverage proprietary data, user-provided documents, or any other textual information to provide accurate and context-rich responses. OpenAI handles the heavy lifting of processing, embedding, and searching your files, so you can focus on building amazing applications.

## Core Concepts

### 1. Vector Stores: Your Assistant's Knowledge Library

*   **Vector Stores** are specialized databases designed for efficient storage and retrieval of information from your files.
*   Files added to a Vector Store are automatically processed (parsed, chunked, embedded) to enable both keyword and semantic search.
*   Each Vector Store can store up to 10,000 files.
*   You can attach Vector Stores to both Assistants and Threads.

### 2. File Search in Action

1.  **Upload:** Upload your files to OpenAI.
2.  **Store:** Create a Vector Store and add your uploaded files.
3.  **Attach:** Connect the Vector Store to your Assistant or Thread.
4.  **Query:** The Assistant intelligently uses File Search when needed, based on user queries.
5.  **Retrieve & Respond:** The Assistant retrieves relevant information from the Vector Store and uses it to craft a response.

## Example: Building a Research Assistant for Physics Papers

Let's build a research assistant that can answer questions based on physics research papers from arXiv.

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI
import time

# Initialize the OpenAI client
client = OpenAI()

### Step 1: Create an Assistant with File Search

In [ ]:
assistant = client.beta.assistants.create(
    name="Physics Research Assistant",
    instructions="You are a helpful research assistant. Use the provided physics papers to answer questions about the latest research.",
    model="gpt-4o",
    tools=[{"type": "file_search"}],
)

### Step 2: Download Research Papers and Create a Vector Store

We'll use two example papers from arXiv. You can download them directly using their URLs.

In [ ]:
import requests

# Download example physics papers from arXiv
paper_urls = [
    "[invalid URL removed]",  # Example paper 1
    "[invalid URL removed]",  # Example paper 2
]
file_paths = []

for i, url in enumerate(paper_urls):
    response = requests.get(url)
    response.raise_for_status()  # Ensure the download was successful
    file_path = f"arxiv_paper_{i+1}.pdf"
    with open(file_path, "wb") as f:
        f.write(response.content)
    file_paths.append(file_path)

# Create a vector store for research papers
vector_store = client.beta.vector_stores.create(name="Physics Research Papers")

# Prepare the files for upload to OpenAI
file_streams = [open(path, "rb") for path in file_paths]

# Upload files, add them to the vector store, and poll for completion
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

print(f"File batch status: {file_batch.status}")
print(f"File counts: {file_batch.file_counts}")

**Important:** Always ensure all files are fully processed (`status: "completed"`) before creating runs.

### Step 3: Connect the Vector Store to Your Assistant

In [ ]:
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

### Step 4: Create a Thread and Ask a Question

In [ ]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are the main findings of the paper about causal discovery?",
)

### Step 5: Run the Assistant and Observe File Search in Action

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI

client = OpenAI()

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nAssistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nAssistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        if citations:
            print("\nCitations:")
            print("\n".join(citations))

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please cite the source paper when answering.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

When you run this code, you'll see the assistant using `file_search` and providing an answer based on the uploaded research papers, citing the source document.

## Customizing File Search

### Chunking

*   **Default:** Chunks of 800 tokens with a 400-token overlap.
*   **Customize:** Adjust `max_chunk_size_tokens` (100-4096) and `chunk_overlap_tokens` (0 to `max_chunk_size_tokens / 2`) when adding files to the Vector Store.

### Number of Chunks

*   **Default:** Up to 20 chunks for `gpt-4*` models, 5 for `gpt-3.5-turbo`.
*   **Customize:** Set `file_search.max_num_results` when creating the assistant or run.
*   **Note:** Fewer chunks might be returned due to token budget limits or if fewer chunks are available.

### Improving Relevance with Ranking

*   **Problem:** Low-relevance results can degrade response quality.
*   **Solution:** Adjust the ranker settings.
    *   `ranker`: Choose the ranking algorithm (`auto` or `default_2024_08_21`).
    *   `score_threshold`: Set a minimum relevance score (0.0-1.0) to filter out low-relevance chunks.

### Inspecting Search Results
Use the `include` parameter in the API to retrieve the specific content used to generate the answer:

In [ ]:
run_step = client.beta.threads.runs.steps.retrieve(
    thread_id=thread.id,
    run_id=run.id,
    step_id=step.id,
    include=["step_details.tool_calls[*].file_search.results[*].content"]
)

## Managing Costs with Expiration Policies

*   **Vector Store Costs:** You're billed based on the size of your Vector Stores ($0.10/GB/day after the first free GB).
*   **Expiration Policies:** Control costs by setting expiration policies when creating or updating Vector Stores.
*   **Default for Thread Vector Stores:** 7 days of inactivity.

In [ ]:
vector_store = client.beta.vector_stores.create(
    name="Temporary Store",
    expires_after={"anchor": "last_active_at", "days": 3}, # Expires after 3 days of inactivity
)

## Supported File Types

File Search supports a wide range of text-based file formats, including:

*   `.pdf`, `.txt`, `.md`, `.docx`, `.html`, `.json`, `.js`, `.py`, and many more (see the documentation for a complete list).
*   **Encoding:** `utf-8`, `utf-16`, or `ascii`.
*   **Maximum File Size:** 512 MB.
*   **Maximum Tokens per File:** 5,000,000.

## Conclusion

File Search empowers your Assistants with the ability to leverage your data, transforming them into powerful tools for research, customer support, analysis, and more. By understanding Vector Stores, customizing settings, and managing costs effectively, you can build sophisticated, knowledge-rich assistants that meet your specific needs.